<a href="https://colab.research.google.com/github/pankrit9/AttentionSnatcher/blob/main/recipe_image_gen_bean_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q  transformers torch accelerate tqdm pillow
!pip install -q -U huggingface_hub diffusers


# Supabase and cloudinary setup

In [ ]:
!pip install -q supabase cloudinary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.3/147.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 18.8 MB/s eta 0:00:00


In [ ]:
import torch
from diffusers import DiffusionPipeline
from supabase import create_client, Client
from google.colab import userdata
import cloudinary
import cloudinary.uploader
import os
import time
from tqdm import tqdm
import io


In [ ]:

SUPABASE_URL = userdata.get("SUPABASE_URL")
SUPABASE_KEY = userdata.get("SUPABASE_KEY")

def initialize_supabase():
    """Initialize Supabase client with proper error handling"""
    global supabase

    if not SUPABASE_URL or not SUPABASE_KEY:
        print("Missing Supabase credentials. Set SUPABASE_URL and SUPABASE_KEY environment variables", "ERROR")
        raise ValueError("Missing Supabase credentials")

    try:
        print("Initializing Supabase client...", "INFO")
        supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
        print("Supabase client initialized successfully", "SUCCESS")

        return supabase
    except Exception as e:
        print(f"Failed to initialize Supabase client: {str(e)}", "ERROR")
        raise
initialize_supabase()


Initializing Supabase client... INFO
Supabase client initialized successfully SUCCESS


In [ ]:


CLOUD_NAME = userdata.get("CLOUD_NAME")
CLOUDINARY_KEY = userdata.get("CLOUDINARY_KEY")
CLOUDINARY_SECRET = userdata.get("CLOUDINARY_SECRET")


cloudinary.config(
    cloud_name=CLOUD_NAME,
    api_key=CLOUDINARY_KEY,
    api_secret=CLOUDINARY_SECRET
)

In [ ]:
def upload_to_cloudinary(image, recipe_id):
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='JPEG')
    img_byte_arr = img_byte_arr.getvalue()

    # Upload to Cloudinary
    result = cloudinary.uploader.upload(
        img_byte_arr,
        public_id=f"recipe_{recipe_id}",
        folder="recipe_images"
    )

    return result['secure_url']


def get_recipes_without_images(batch_size=10, offset=0):
    response = supabase.table('Recipe') \
        .select('id, title, description') \
        .is_('image', None) \
        .neq('title', None) \
        .neq('description', None) \
        .limit(batch_size) \
        .offset(offset) \
        .execute()

    return response.data


def update_recipe_image(recipe_id, image_url):
    supabase.table('Recipe') \
        .update({'image': image_url,'imageType':'jpg'}) \
        .eq('id', recipe_id) \
        .execute()


In [ ]:
def generate_recipe_image(title, description):
    # Create a prompt based on recipe title and description
    prompt=generate_image_prompt(title,description)
    print(f"Generating image for recipe: {title} with prompt {prompt}")

    # Generate the image
    image = pipe(
        prompt=prompt,
    ).images[0]

    return image

def generate_image_prompt(title, description):
    return (
        f"Cooked dish on a table with only the plated food. "
        f"Close-up of only this dish plated well. Dish: {title}. {description}. "
        f"Ultra-realistic, authentic, restaurant style, centered, clean plate, natural light."
    )



In [ ]:
def process_recipes_in_batches():
    offset = 0
    batch_size = 10
    total_processed = 0

    while True:
        # Get batch of recipes without images
        recipes = get_recipes_without_images(batch_size, offset)

        if not recipes:
            print("No more recipes to process!")
            break

        print(f"Processing batch of {len(recipes)} recipes...")

        for recipe in tqdm(recipes):
            try:
                # Generate image
                image = generate_recipe_image(recipe['title'], recipe['description'])

                # Upload to Cloudinary
                image_url = upload_to_cloudinary(image, recipe['id'])

                # Update recipe in Supabase
                update_recipe_image(recipe['id'], image_url)

                total_processed += 1
                print(f"Updated recipe {recipe['id']} with image URL: {image_url}")
                print(f"{image_url}")

                # Small delay to avoid rate limits
                time.sleep(1)

            except Exception as e:
                print(f"Error processing recipe {recipe['id']}: {e}")

        # Move to next batch
        offset += batch_size

    print(f"Total recipes processed: {total_processed}")

In [ ]:
import torch
from diffusers import DiffusionPipeline

model_id = "RunDiffusion/Juggernaut-XL-v9"
print(f"Loading model: {model_id} with fp16 precision...")

# Create the pipeline with fp16 precision
pipe = DiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)

pipe = pipe.to("cuda")

Loading model: RunDiffusion/Juggernaut-XL-v9 with fp16 precision...


model_index.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
process_recipes_in_batches()


Processing batch of 10 recipes...


  0%|          | 0/10 [00:00<?, ?it/s]

Generating image for recipe: Cola Cake with prompt Cooked dish on a table with only the plated food. Close-up of only this dish plated well. Dish: Cola Cake. A moist and delicious Cola Cake with a rich frosting. This recipe combines the unique flavor of cola with classic baking ingredients for a crowd-pleasing dessert.. Ultra-realistic, authentic, restaurant style, centered, clean plate, natural light.


  0%|          | 0/50 [00:00<?, ?it/s]

Updated recipe ccb320bf-087c-4026-af9b-b40de74de3a2 with image URL: https://res.cloudinary.com/dpvvnl2vg/image/upload/v1748317267/recipe_images/recipe_ccb320bf-087c-4026-af9b-b40de74de3a2.jpg
https://res.cloudinary.com/dpvvnl2vg/image/upload/v1748317267/recipe_images/recipe_ccb320bf-087c-4026-af9b-b40de74de3a2.jpg


...

  0%|          | 0/50 [00:00<?, ?it/s]

Updated recipe 5a79f830-4da7-4280-8de9-cf6aaf010ed0 with image URL: https://res.cloudinary.com/dpvvnl2vg/image/upload/v1748319333/recipe_images/recipe_5a79f830-4da7-4280-8de9-cf6aaf010ed0.jpg
https://res.cloudinary.com/dpvvnl2vg/image/upload/v1748319333/recipe_images/recipe_5a79f830-4da7-4280-8de9-cf6aaf010ed0.jpg


 50%|█████     | 5/10 [03:52<03:52, 46.45s/it]The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['.']


Generating image for recipe: Tofu Salad with prompt Cooked dish on a table with only the plated food. Close-up of only this dish plated well. Dish: Tofu Salad. A quick and easy vegan tofu salad, perfect as a sandwich filling or served on greens. Packed with protein and flavor, it's a healthy and satisfying meal option.. Ultra-realistic, authentic, restaurant style, centered, clean plate, natural light.


  0%|          | 0/50 [00:00<?, ?it/s]

================================================================
================================================================

# Image gen testing

================================================================
================================================================

In [ ]:
import torch
from diffusers import DiffusionPipeline

In [ ]:
model_id = "RunDiffusion/Juggernaut-XL-v9"
print(f"Loading model: {model_id} with fp16 precision...")

# Create the pipeline with fp16 precision
pipe = DiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)

pipe = pipe.to("cuda")


Loading model: RunDiffusion/Juggernaut-XL-v9 with fp16 precision...


model_index.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
image__gen_1="Cooked dish plated on a dining table and nothing else in the image, just the plated food. It should be a close up image of the dish with barely anything else visible. There should be only one dish in one image and one plate only. The dish is Pan-Seared Salmon with Lemon Butter Sauce. Here is a short description of the recipe: A tender salmon fillet seared to crispy-skin perfection, served on a bed of roasted asparagus and baby potatoes, drizzled with a tangy lemon butter sauce and garnished with fresh parsley. Strictly follow the following instructions while generating images: The image should strictly be as realistic as possible. The image should be from the top view of the plated dish. The dish should be in the center of the image. It should look like a dish cooked in a restaurant and plated on a clean plate with studio light."



image_gen_2="Highly realistic, top-view image of a single plated dish only. Dish centered on a clean white plate. Studio lighting, no background, no props, no table, no hands, nothing else visible. Close-up restaurant-style presentation. Pan-Seared Salmon with lemon butter sauce, asparagus, baby potatoes, and parsley garnish."

image_gen_3="Ultra-realistic, close-up image of a single plated dish only. Dish centered on a clean white plate, placed on a plain wooden table. Studio lighting, no hands, no props, no distractions. Magazine-quality restaurant presentation. Pan-Seared Salmon with lemon butter sauce, asparagus, baby potatoes, and parsley garnish."

image_gen_4="Highly-realistic, close-up image of a single plated dish only. Dish centered on a clean white plate, placed on a plain wooden table. Studio lighting, no hands, no props, no distractions. Magazine-quality restaurant presentation. Creamy Garlic Tuscan Chicken with rich parmesan sauce, sun-dried tomatoes, and wilted spinach."

image_gen_5="Cooked dish plated on a dining table and nothing else in the image, just the plated food. It should be a close-up image of the dish. Only one plate, one dish. Strictly realistic, studio-lit like restaurant food. Pan-Seared Salmon with crispy skin, on roasted asparagus and baby potatoes, lemon butter sauce, parsley garnish. Clean plate."

image_gen_6 ="Cooked dish plated on a dining table and nothing else in the image, just the plated food. It should be a close up image of the dish with barely anything else visible. There should be only one dish in one image and one plate only. The dish is Burritos. Here is a short description of the recipe: Warm, soft flour tortillas filled with seasoned grilled chicken, black beans, rice, fresh salsa, and melted cheese, served with a side of guacamole. Strictly follow the following instructions while generating images: The image should strictly be as realistic as possible. The image should be from the top view of the plated dish. The dish should be in the center of the image. It should look like a dish cooked in a restaurant and plated on a clean plate with studio light."

image_gen_7="Cooked dish plated on a wooden table with nothing else visible, just the plated food. Close-up image of one dish on one plate only. The dish is Creamy Alfredo Pasta. Al dente fettuccine in rich parmesan sauce, garnished with parsley and cracked pepper. Must be ultra-realistic, restaurant style, centered, clean plate, natural lighting."

image_gen_8="Cooked dish plated on a wooden table with nothing else visible, just the plated food. Close-up image of one dish on one plate only. The dish is Biryani. Fragrant basmati rice cooked with tender spiced chicken, caramelized onions, and fresh herbs. Must be ultra-realistic, restaurant style, centered, clean plate, natural lighting."

image_gen_10="Cooked dish plated on a wooden table with nothing else visible, just the plated food. Close-up image of one dish on one plate only. The dish is Chickpea Curry with Rice. Creamy spiced chickpea curry served with fluffy white rice and fresh cilantro garnish. Must be ultra-realistic, authentic, restaurant style, centered, clean plate, natural lighting."

image_gen_11="Cooked dish plated on a wooden table with nothing else visible, just the plated food. Close-up image of one dish on one plate only. The dish is Rainbow Salad. A vibrant mix of fresh lettuce, cherry tomatoes, cucumber, bell peppers, shredded carrots, and purple cabbage, topped with a light vinaigrette. Must be ultra-realistic, authentic, restaurant style, centered, clean plate, natural lighting."

In [ ]:
image_1="Cooked dish plated on a wooden table with nothing else visible, just the plated food. Close-up image of one dish on one plate only. The dish is White Sauce Pasta. Tender pasta tossed in creamy white sauce with garlic and herbs, garnished with parsley. Must be ultra-realistic, authentic, restaurant style, centered, clean plate, natural lighting."

image_2="Cooked dish plated on a wooden table with nothing else visible, just the plated food. Close-up image of one dish on one plate only. The dish is Seafood Paella. Saffron-infused rice with shrimp, mussels, squid, and peas, garnished with lemon wedges. Must be ultra-realistic, authentic, restaurant style, centered, clean plate, natural lighting."

image_3="Cooked dish plated on a wooden table with nothing else visible, just the drink in a glass. Close-up image of one glass only. The drink is Fresh Mango Juice. Bright golden mango juice served in a clear glass with condensation, garnished with a mint leaf. Must be ultra-realistic, authentic, restaurant style, centered, clean glass, natural lighting."

image_4="Cooked dish plated on a wooden table with nothing else visible, just the plated food. Close-up image of one dish on one plate only. The dish is Chicken Sandwich. Grilled chicken breast with lettuce, tomato, and mayo on toasted bread, garnished with a pickle. Must be ultra-realistic, authentic, restaurant style, centered, clean plate, natural lighting."

image_5="Cooked dish plated on a wooden table with nothing else visible, just the plated food. Close-up image of one dish on one bowl only. The dish is Dal Fry. Spiced yellow lentils tempered with mustard seeds, garlic, and fresh cilantro garnish. Must be ultra-realistic, authentic, restaurant style, centered, clean bowl, natural lighting."

image_6="Cooked dish plated on a wooden table with nothing else visible, just the plated food. Close-up image of one dish on one plate only. The dish is Poutine. Crispy golden fries topped with melted cheese curds and rich brown gravy. Must be ultra-realistic, authentic, restaurant style, centered, clean plate, natural lighting."

In [ ]:

prompt="Cooked dish plated on a dining table and nothing else in the image, just the plated food. It should be a close up image of the dish with barely anything else visible. There should be only one dish in one image and one plate only. The dish is Pan-Seared Salmon with Lemon Butter Sauce. Here is a short description of the recipe: A tender salmon fillet seared to crispy-skin perfection, served on a bed of roasted asparagus and baby potatoes, drizzled with a tangy lemon butter sauce and garnished with fresh parsley. Strictly follow the following instructions while generating images: The image should strictly be as realistic as possible. The image should be from the top view of the plated dish. The dish should be in the center of the image. It should look like a dish cooked in a restaurant and plated on a clean plate with studio light."
print(f"Generating image with prompt: {prompt}")



Generating image with prompt: Cooked dish plated on a dining table and nothing else in the image, just the plated food. It should be a close up image of the dish with barely anything else visible. There should be only one dish in one image and one plate only. The dish is Pan-Seared Salmon with Lemon Butter Sauce. Here is a short description of the recipe: A tender salmon fillet seared to crispy-skin perfection, served on a bed of roasted asparagus and baby potatoes, drizzled with a tangy lemon butter sauce and garnished with fresh parsley. Strictly follow the following instructions while generating images: The image should strictly be as realistic as possible. The image should be from the top view of the plated dish. The dish should be in the center of the image. It should look like a dish cooked in a restaurant and plated on a clean plate with studio light.


In [ ]:
image = pipe(
    prompt=image_6,
).images[0]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
image_path = "test_image_6_a.jpg"
image.save(image_path)
print(f"Image saved to {image_path}")

Image saved to test_image_6_a.jpg
